In [1]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


: 

In [2]:
!pip install -q torch torchvision timm

In [3]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import timm
import json
from pathlib import Path
from tqdm import tqdm

DATA_DIR = Path("/content/drive/MyDrive/data/processed")
OUTPUT_DIR = Path("/content/drive/MyDrive/models")
OUTPUT_DIR.mkdir(exist_ok=True)

BATCH_SIZE = 32
NUM_EPOCHS = 15
LEARNING_RATE = 1e-4
IMG_SIZE = 224
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device: {DEVICE}")

Using device: cuda


## Load dataset info

In [6]:
with open(DATA_DIR / "dataset_info.json") as f:
    dataset_info = json.load(f)

NUM_CLASSES = len(dataset_info["categories"])
CLASS_NAMES = [c.lower().replace(" ", "_") for c in dataset_info["categories"]]
print(f"Classes: {CLASS_NAMES}")

Classes: ['shirts', 'watches', 'casual_shoes', 'tops', 'handbags']


## Data loading

In [ ]:
train_transform = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(15),
        transforms.ColorJitter(brightness=0.2, contrast=0.2),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

val_transform = transforms.Compose(
    [
        transforms.Resize((IMG_SIZE, IMG_SIZE)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

train_dataset = datasets.ImageFolder(DATA_DIR / "train", transform=train_transform)
val_dataset = datasets.ImageFolder(DATA_DIR / "val", transform=val_transform)
test_dataset = datasets.ImageFolder(DATA_DIR / "test", transform=val_transform)

train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2
)
val_loader = DataLoader(
    val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2
)
test_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2
)

print(
    f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}"
)
print(f"Class mapping: {train_dataset.class_to_idx}")

Train: 1750, Val: 375, Test: 375
Class mapping: {'casual shoes': 0, 'handbags': 1, 'shirts': 2, 'tops': 3, 'watches': 4}


: 

## Model Setup

In [ ]:
model = timm.create_model("efficientnet_b0", pretrained=True, num_classes=NUM_CLASSES)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode="max", patience=2, factor=0.5
)

## Training Loop

In [ ]:
history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}
best_val_acc = 0.0

for epoch in range(NUM_EPOCHS):
    # Training
    model.train()
    train_loss, train_correct, train_total = 0, 0, 0

    for images, labels in tqdm(
        train_loader, desc=f"Epoch {epoch + 1}/{NUM_EPOCHS} [Train]"
    ):
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        train_total += labels.size(0)
        train_correct += predicted.eq(labels).sum().item()

    train_loss = train_loss / len(train_loader)
    train_acc = 100.0 * train_correct / train_total

    # Validation
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0

    with torch.no_grad():
        for images, labels in tqdm(
            val_loader, desc=f"Epoch {epoch + 1}/{NUM_EPOCHS} [Val]"
        ):
            images, labels = images.to(DEVICE), labels.to(DEVICE)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_loss = val_loss / len(val_loader)
    val_acc = 100.0 * val_correct / val_total

    history["train_loss"].append(train_loss)
    history["train_acc"].append(train_acc)
    history["val_loss"].append(val_loss)
    history["val_acc"].append(val_acc)

    print(
        f"Epoch {epoch + 1}: Train Loss={train_loss:.4f}, Train Acc={train_acc:.2f}%, Val Loss={val_loss:.4f}, Val Acc={val_acc:.2f}%"
    )

    scheduler.step(val_acc)

    # Save best model
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(
            {
                "epoch": epoch,
                "model_state_dict": model.state_dict(),
                "optimizer_state_dict": optimizer.state_dict(),
                "val_acc": val_acc,
                "class_to_idx": train_dataset.class_to_idx,
            },
            OUTPUT_DIR / "best_model.pth",
        )
        print(f"✓ Saved best model (Val Acc: {val_acc:.2f}%)")

print(f"\nBest validation accuracy: {best_val_acc:.2f}%")

## Test evaluation

In [ ]:
checkpoint = torch.load(OUTPUT_DIR / "best_model.pth")
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

test_correct, test_total = 0, 0
with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing"):
        images, labels = images.to(DEVICE), labels.to(DEVICE)
        outputs = model(images)
        _, predicted = outputs.max(1)
        test_total += labels.size(0)
        test_correct += predicted.eq(labels).sum().item()

test_acc = 100.0 * test_correct / test_total
print(f"Test Accuracy: {test_acc:.2f}%")

## Save training metadata

In [ ]:
metadata = {
    "model_architecture": "efficientnet_b0",
    "num_classes": NUM_CLASSES,
    "class_names": CLASS_NAMES,
    "class_to_idx": train_dataset.class_to_idx,
    "img_size": IMG_SIZE,
    "batch_size": BATCH_SIZE,
    "num_epochs": NUM_EPOCHS,
    "learning_rate": LEARNING_RATE,
    "best_val_acc": best_val_acc,
    "test_acc": test_acc,
    "history": history,
}

with open(OUTPUT_DIR / "training_metadata.json", "w") as f:
    json.dump(metadata, f, indent=2)

print("✓ Training complete. Files saved to:", OUTPUT_DIR)
print("  - best_model.pth")
print("  - training_metadata.json")

## Plot training curves

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

ax1.plot(history['train_loss'], label='Train Loss')
ax1.plot(history['val_loss'], label='Val Loss')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.set_title('Training and Validation Loss')

ax2.plot(history['train_acc'], label='Train Acc')
ax2.plot(history['val_acc'], label='Val Acc')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy (%)')
ax2.legend()
ax2.set_title('Training and Validation Accuracy')

plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'training_curves.png', dpi=150, bbox_inches='tight')
plt.show()